# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of [ABBA with python](https://github.com/BIOP/abba_python)

For this notebook to work, you will as well need to:
* set the elastix and transformix path for the elastix registration steps.
* to have DeepSlice installed in a separate conda environment (pip installed deepslice) and set its path
These path will be stored for any future abba instance created (stored in user Java prefs)

By running this notebook, you will perform a fully automated registration of demo mouse brain sections to the Adult Mouse Allen Brain atlas.

If this is the first, you will need an internet connection, as the atlas will be downloaded, as well as some sample dataset.

The dataset is downloaded from https://zenodo.org/record/6592478

Multichannel registration works reasonably well in this notebook because the first channel of the atlas (NISSL, indexed 0), is ressembling the DAPI channel of the slices (indexed 0), and the second channel of the atlas (ARA, indexed 1), is ressembling the autofluorescence channel of the slices (indexed 1).

In [ ]:
# core dependencies
import os
import time
from pathlib import Path

basePath = os.getcwd() + '/images/'

In [ ]:
# for testing in IDE - not required if you installed abba_python via pip!
# os.chdir('../src')
# os.getcwd()

In [ ]:
from abba_python import abba

from bg_atlasapi import utils

#from scijava_python_command.magic import cell_with_modal_ui

## Download test sections if necessary

In [ ]:
# Demo dataset for automated slices registration
zenodo_demo_slices_url = 'https://zenodo.org/record/6592478/files/'


# Only one section every five section is used for this demo
demo_sections = [
    'S00.tif',
    'S05.tif',
    'S10.tif',
    'S15.tif',
    'S20.tif',
    'S25.tif',
    'S30.tif',
    'S35.tif',
    'S40.tif',
    'S45.tif',
    'S50.tif',
    'S55.tif',
    'S60.tif',
    'S65.tif',
    'S70.tif',
    'S75.tif',
    'S80.tif']


def download_if_necessary(base_path, section_name):
    output_path = Path(base_path + section_name)
    if not output_path.exists():
        utils.check_internet_connection()
        url = zenodo_demo_slices_url + section_name + '?download=1'
        utils.retrieve_over_http(url, output_path)


def download_test_images(base_path):
    [download_if_necessary(base_path, section) for section in demo_sections]

if not os.path.exists(basePath):
    os.makedirs(basePath)
    
download_test_images(basePath)

## ABBA initialization

In [ ]:
headless = False
from abba_python.abba import Abba

# -- FOR DEBUGGING
# import imagej.doctor
# imagej.doctor.checkup()
# imagej.doctor.debug_to_stderr()

import logging
# logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.WARN)

if headless:
    # -- HEADLESS
    # create a thread: the jupyter UI will not be responsive is the cell is not finished. 
    # that's why it's needed to split the initialisation in two cells.
    aligner = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1', headless=True)
    
else:
    # -- NOT HEADLESS
    aligner = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1')
    aligner.show_bdv_ui()  # creates and show a bdv view



In [ ]:
# (Optional cell) DO NOT MOVE THIS PIECE OF CODE HIGHER UP: Java needs to be initialised
# Goal: make java less talkative
from scyjava import jimport
DebugTools = jimport('loci.common.DebugTools')
DebugTools.enableLogging("DEBUG")

## Setup ABBA: elastix, transformix, and DeepSlice

In [ ]:
# Optional: run this cell if you need to setup ABBA:
# - set up elastix and transformix
# - set up deepslice extra conda environment (required for this notebook)
# - set up a different place to store the atlases
if not headless:
    # Maybe just run it once not in headless to set elastix and transformix path correctly. They will be remembered in headless mode (stored in java Prefs)
    aligner.ij.command().run('ch.epfl.biop.wrappers.ij2command.BiopWrappersSet',True)
    # Maybe just run it once not in headless to set the DeepSlice Conda env correctly
    aligner.ij.command().run('ch.epfl.biop.wrappers.deepslice.ij2commands.DeepSlicePrefsSet',True)
else:
    pass
    # If you want to set elastix and transformix Path without GUI, you can comment 'pass' and set the 
    # proper exe paths below:
    aligner.set_deepslice_env('C:/Users/nico/.conda/envs/deep_slice', '1.1.5')
    aligner.set_elastix_path('C:/elastix-5.0.1-win64/elastix.exe')
    aligner.set_transformix_path('C:/elastix-5.0.1-win64/transformix.exe')
    # aligner.set_atlas_cache_dir('path_to_cache_dir') # Uncomment if you want to change the default location for caching the atlases
    
aligner.print_config()

## 3. Import sections into ABBA

In [ ]:
# NOTE! Remove the .bfmemo file in the images folder! (cf https://github.com/ome/bioformats/issues/3957)

# import sections into ABBA
files = [basePath + section for section in demo_sections]
aligner.import_from_files(filepaths=files)

# ALL REGISTRATIONS AND COMMANDS BELOW ARE PERFORMED ON THE SELECTED SLICES!!
# since we want to register all of them, we select all of them
# abba.select_all_slices()
aligner.deselect_all_slices()
# abba.select_slices([0]) # only one for testing
# abba.select_all_slices()


In [ ]:
aligner.select_all_slices()

In [ ]:
# we want to avoid saturation in the display. This does not matter for
# all registration methods EXCEPT for DeepSlice, which takes in rgb images
aligner.change_display_settings(0, 0, 500)
aligner.change_display_settings(1, 0, 1200)

if not headless:
    # programmatic way to show (or hide) sections and channels
    aligner.get_bdv_view().setSelectedSlicesVisibility(True)
    aligner.get_bdv_view().setSelectedSlicesVisibility(0, True) # Channel 0
    aligner.get_bdv_view().setSelectedSlicesVisibility(1, True) # Channel 1

## 4. DeepSlice Registration(s)

In [ ]:
# a first deepslice registration round : possible because it's the Allen CCF atlas, cut in coronal mode
# what's assumed : the sections are already in the correct order
aligner.register_slices_deepslice_local(channels='0,1',
                                  ensemble=False,
                                  model='mouse',
                                  post_processing='Keep order + ensure regular spacing',
                                  slices_spacing_micrometer=-1,      
                                  allow_slicing_angle_change=True) # use run_mode='Web' if you are not headless and without a local deepslice env

#allow_slicing_angle_change: bool,
#                                        channels: str,
#                                        ensemble: bool,
#                                        model: str,
#                                        post_processing: str,
#                                        slices_spacing_micrometer: float

# second deepslice registration: because the slices are resampled for the registration,
# we usually get a slightly better positioning along z and cutting angle
# also: it's fast, and the combination of two affine transforms is
# an affine transform, so it's not like we are adding extra degrees of freedom
# aligner.register_slices_deepslice(channels=[0, 1],
#                                  maintain_slices_order = True, # the slices are already sorted: do not mess the order
#                                  run_mode='local') # use run_mode='web' if you are not headless and without a local deepslice env

## 5. Elastix affine registration

In [ ]:

# a round of elastix registration, affine
# the channel 0 of the dataset (DAPI) is registered with the Nissl Channel of the atlas (0)
# and the channel 1 of the dataset (mainly autofluo) is registered with the autofluo channel of the atlas (1)
# these two channels have equal weights in the registration process
aligner.register_slices_elastix_affine(channels_slice_csv='0,1',
                                    channels_atlas_csv='0,1',
                                    pixel_size_micrometer=40)

## 6. Elastix spline registration

In [ ]:
# %%cell_with_modal_ui
# optional: a round of elastix registration, spline
# same channels as in the affine registration
# 5 control points along x = very coarse spline (and thus maybe unnecessary)
# abba.register_elastix_spline(
#    nb_control_points=5,
#    atlas_image_channels=[0, 1],
#    slice_image_channels=[0, 1],
#    pixel_size_micrometer=40).get()

# a round of elastix registration, affine
# same channels as in the affine registration 
# 16 control points = reasonable spline, which allows for local corrections, without deforming two much the section
aligner.register_slices_elastix_spline(channels_slice_csv='0,1',
                                    channels_atlas_csv='0,1',
                                    nb_control_points_x=16,
                                    pixel_size_micrometer=20)

## 7. Wait for end of all registrations

In [ ]:
# all tasks/registrations are enqueued and executed asynchronously
# if you need to wait before saving, then wait for all tasks to be finished:
aligner.wait_for_end_of_tasks()

## 8. Saving the result

In [ ]:
# abba.set_slices_thickness_match_neighbors() # not critical, but for 3d reconstruction it will allow for each slice to occupy the place available between its neighbors

save_dir = os.path.join(os.getcwd(), 'temp', 'notebook0', 'state')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


aligner.state_save(save_dir+"/state.abba") # full absolute path needed, returns True if the process was successfull - it won't erase an existing file